In [1]:
import requests
import json
import pandas as pd
import importlib
from torch import nn
from datetime import datetime, timezone, timedelta
from torch.utils.data import DataLoader
import utils
importlib.reload(utils)
from utils import *
import dataloaders
importlib.reload(dataloaders)
from dataloaders import *
import pickle



import dataloaders
importlib.reload(dataloaders)
from dataloaders import *

import model_definitions
importlib.reload(model_definitions)
from model_definitions import NeuralNetwork

API_KEY = 'c82bb85aa6c7754bb3c336b3936dd983'
APPROVED_SPORTSBOOKS = [ 'DraftKings', 'PointsBet (US)',]
# APPROVED_SPORTSBOOKS = [ 'DraftKings']

def get_current_odds():
    url = f'https://api.the-odds-api.com/v4/sports/basketball_nba/odds/?apiKey={API_KEY}&regions=us&markets=h2h'
    response = requests.get(url)
    data = json.loads(response.text)
    return data

def clean_data(data):
    # Only keep odds from approved sportsbooks
    for game in data:
        game['bookmakers'] = [book for book in game['bookmakers'] if book['title'] in APPROVED_SPORTSBOOKS]
    # Only keep games with odds for games that commence today
    data = [game for game in data if datetime.fromisoformat(game['commence_time']).astimezone(timezone(timedelta(hours=-5))).date() == datetime.now().astimezone(timezone(timedelta(hours=-5))).date()]
    # Only keep odds for games that haven't started yet
    data = [game for game in data if datetime.fromisoformat(game['commence_time']).astimezone(timezone.utc) > datetime.now().astimezone(timezone.utc)]
    # Only keep games with odds from at least 1 sportsbook
    data = [game for game in data if len(game['bookmakers']) > 0]
    
    return data

def find_best_odds(data):
    best_odds = []
    for game in data:
        commence_time = game['commence_time']
        home_team = game['home_team']
        away_team = game['away_team']
        best_home_odds = max([(book['title'], book['markets'][0]['outcomes'][i]['price']) for book in game['bookmakers'] for i in range(2) if book['markets'][0]['outcomes'][i]['name'] == home_team], key=lambda x: x[1])
        best_away_odds = max([(book['title'], book['markets'][0]['outcomes'][i]['price']) for book in game['bookmakers'] for i in range(2) if book['markets'][0]['outcomes'][i]['name'] == away_team], key=lambda x: x[1])
        best_odds.append({
            'home_team': home_team,
            'away_team': away_team,
            'best_home_odds': best_home_odds[1],
            'best_home_bookmaker': best_home_odds[0],
            'best_away_odds': best_away_odds[1],
            'best_away_bookmaker': best_away_odds[0],
            'commence_time': commence_time,
        })
    df = pd.DataFrame(best_odds)
    print(df)
    df['commence_time'] = pd.to_datetime(df['commence_time'])
    df['commence_time'] = df['commence_time'].dt.tz_convert('US/Eastern')
    return df

In [2]:
data = get_current_odds()
data = clean_data(data)
best_odds = find_best_odds(data)
best_odds

                 home_team             away_team  best_home_odds  \
0       Washington Wizards         Chicago Bulls            1.80   
1       Philadelphia 76ers         Orlando Magic            1.34   
2           Boston Celtics     Charlotte Hornets            1.34   
3          New York Knicks         Brooklyn Nets            1.16   
4      Cleveland Cavaliers        Indiana Pacers            1.74   
5   Minnesota Timberwolves         Atlanta Hawks            1.10   
6        San Antonio Spurs        Denver Nuggets            5.10   
7        Memphis Grizzlies    Los Angeles Lakers            9.00   
8               Miami Heat       Toronto Raptors            1.10   
9    Oklahoma City Thunder       Milwaukee Bucks            1.08   
10        Dallas Mavericks       Detroit Pistons            1.11   
11   Golden State Warriors  New Orleans Pelicans            1.56   
12  Portland Trail Blazers       Houston Rockets            3.00   
13    Los Angeles Clippers             Utah Jazz

,home_team,away_team,best_home_odds,best_home_bookmaker,best_away_odds,best_away_bookmaker,commence_time
0,Washington Wizards,Chicago Bulls,1.80,DraftKings,2.10,PointsBet (US),2024-04-12 19:00:00-04:00
1,Philadelphia 76ers,Orlando Magic,1.34,DraftKings,3.40,DraftKings,2024-04-12 19:00:00-04:00
2,Boston Celtics,Charlotte Hornets,1.34,PointsBet (US),3.35,DraftKings,2024-04-12 19:30:00-04:00
3,New York Knicks,Brooklyn Nets,1.16,DraftKings,5.55,DraftKings,2024-04-12 19:40:00-04:00
4,Cleveland Cavaliers,Indiana Pacers,1.74,DraftKings,2.15,PointsBet (US),2024-04-12 19:40:00-04:00
5,Minnesota Timberwolves,Atlanta Hawks,1.10,DraftKings,8.00,PointsBet (US),2024-04-12 20:00:00-04:00
6,San Antonio Spurs,Denver Nuggets,5.10,DraftKings,1.20,PointsBet (US),2024-04-12 20:00:00-04:00
7,Memphis Grizzlies,Los Angeles Lakers,9.00,PointsBet (US),1.08,PointsBet (US),2024-04-12 20:00:00-04:00
8,Miami Heat,Toronto Raptors,1.10,PointsBet (US),8.00,DraftKings,2024-04-12 20:00:00-04:00
9,Oklahoma City Thunder,Milwaukee Bucks,1.08,PointsBet (US),9.00,DraftKings,2024-04-12 20:10:00-04:00


In [3]:
_, player_data, _, team_data, _, injury_report = load_data()

model_version = 'model_deepv4'

optimal_params = get_optimal_params(model_version)
print(optimal_params)

dataset = UpcomingGamesDataset(best_odds, 9, player_data, team_data, model_version, injury_report)

device = 'cpu'

# Load model
num_players = dataset[0][0].shape[1]
num_features = dataset[0][0].shape[2]
num_team_features = dataset[0][1].shape[1]
model = NeuralNetwork(num_players, num_features, num_team_features, hidden_layers=[optimal_params['hidden_layer_0'], optimal_params['hidden_layer_1']])
model.load_state_dict(torch.load(f'../models/{model_version}.pth', map_location=device))
model.to(device)
model.eval()

dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# Make predictions
predictions = {}
for x, x_team, home_team, away_team, in dataloader:
    x = x.float()
    x = x.to(device)
    x_team = x_team.float()
    x_team = x_team.to(device)
    output = nn.Sigmoid()(model(x, x_team))
    output = output.item()
    home_team = home_team[0]
    away_team = away_team[0]
    predictions[(home_team, away_team)] = output

for game, prediction in predictions.items():
    print(f'{prediction} -- {game[0]} vs {game[1]} -- {1 - prediction}')

Could not find player Jaylen Clark
Could not find player D.J. Carton
{'lr': 0.0001838368527157626, 'hidden_layer_0': 189, 'hidden_layer_1': 13, 'stepsize': 7, 'gamma': 0.7234329400126882, 'epochs': 12, 'shuffle': False}
0 (14, 48) (11, 48)
0 (14, 48) (11, 48)
0 (9, 48) (9, 48)
0 (14, 48) (11, 48)
0 (14, 48) (11, 48)
0 (9, 48) (9, 48)
0 (14, 48) (11, 48)
0 (14, 48) (11, 48)
0 (9, 48) (9, 48)
0 (14, 48) (11, 48)
0 (14, 48) (11, 48)
0 (9, 48) (9, 48)
1 (19, 48) (17, 48)
1 (19, 48) (17, 48)
1 (9, 48) (9, 48)
2 (15, 48) (22, 48)
2 (15, 48) (22, 48)
2 (9, 48) (9, 48)
3 (17, 48) (13, 48)
3 (17, 48) (13, 48)
3 (9, 48) (9, 48)
4 (13, 48) (16, 48)
4 (13, 48) (16, 48)
4 (9, 48) (9, 48)
5 (17, 48) (11, 48)
5 (17, 48) (11, 48)
5 (9, 48) (9, 48)
6 (8, 48) (17, 48)
6 (9, 48) (17, 48)
6 (9, 48) (9, 48)
7 (10, 48) (13, 48)
7 (10, 48) (13, 48)
7 (9, 48) (9, 48)
8 (15, 48) (19, 48)
8 (15, 48) (19, 48)
8 (9, 48) (9, 48)
9 (18, 48) (14, 48)
9 (18, 48) (14, 48)
9 (9, 48) (9, 48)
10 (16, 48) (17, 48)
10 (16,

In [4]:
import cvxpy as cp
import numpy as np

# Example data (replace with your actual data)
odds = np.zeros((2*len(best_odds), 2*len(best_odds)))
for i in range(len(best_odds)):
    game = best_odds.iloc[i]
    odds[2*i, 2*i] = game['best_home_odds']
    odds[2*i+1, 2*i+1] = game['best_away_odds']

prob_win = np.zeros(2*len(best_odds))
for i in range(len(best_odds)):
    game = best_odds.iloc[i]
    prob_win[2*i] = predictions[(game['home_team'], game['away_team'])]
    prob_win[2*i+1] = 1 - predictions[(game['home_team'], game['away_team'])]
n_bets = len(odds)  # Number of bets

# Define variables
f = cp.Variable(n_bets)  # Fraction of wealth to bet on each wager

# Calculate the kelly criterion betting fractions
log_returns = cp.multiply(prob_win, cp.log(cp.matmul(odds, f) + 1 - f)) + cp.multiply(1 - prob_win, cp.log(1 - f))

# Objective: Maximize the expected logarithmic return
objective = cp.Maximize(cp.sum(log_returns))

# Constraints: Sum of f <= 1 and all elements of f >= 0
constraints = [cp.sum(f) == 1, f >= 0]

# Define and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS)
print(problem.status)

current_bankroll = 79.92
kelly_fraction = 1/6

wager = current_bankroll * kelly_fraction

print(f'Optimal betting fractions: {f.value}')

print(f'Current bankroll: {current_bankroll:.2f}')
print(f'Amount to bet: {wager:.2f}')

# Optimal betting fractions
optimal_bets = f.value
# Round to 2 decimal places
optimal_bets = np.round(optimal_bets, 2)

bets = pd.DataFrame()
for i in range(len(best_odds)):
    game = best_odds.iloc[i]
    bets = bets._append({
        'home_team': game['home_team'],
        'away_team': game['away_team'],
        'best_home_odds': game['best_home_odds'],
        'best_home_bookmaker': game['best_home_bookmaker'],
        'best_away_odds': game['best_away_odds'],
        'best_away_bookmaker': game['best_away_bookmaker'],
        'optimal_bet_home': round(optimal_bets[2*i] * wager, 2),
        'optimal_bet_away': round(optimal_bets[2*i+1] * wager, 2),
        'home_prediction': prob_win[2*i],
        'away_prediction': prob_win[2*i+1],
    }, ignore_index=True)

# Calculate the expected return
expected_profit = 0
for i in range(len(best_odds)):
    game = best_odds.iloc[i]
    expected_profit += prob_win[2*i] * optimal_bets[2*i] * (game['best_home_odds'] - 1) + (1- prob_win[2*i]) * -optimal_bets[2*i] 
    expected_profit += prob_win[2*i+1] * optimal_bets[2*i+1] * (game['best_away_odds'] - 1) + (1- prob_win[2*i+1]) * -optimal_bets[2*i+1]
    
best_case = 0
for i in range(len(best_odds)):
    game = best_odds.iloc[i]
    price = optimal_bets[2*i] + optimal_bets[2*i+1]
    best_case += max(prob_win[2*i] * (game['best_home_odds'] - 1), prob_win[2*i+1] * (game['best_away_odds'] - 1)) * price
    
print(f'Expected profit: {expected_profit * wager:.2f}', f'({expected_profit * 100:.2f}%)')
print(f'Best case: {best_case * wager:.2f}', f'({best_case * 100:.2f}%)')
bets
    

optimal
Optimal betting fractions: [4.34463643e-13 2.04108017e-01 1.25053602e-11 3.20057973e-10
 2.01388102e-10 5.07441784e-13 6.76681137e-12 1.37894724e-01
 1.67435830e-11 3.29030569e-11 2.33403580e-11 2.76605206e-11
 8.07496486e-11 1.88645911e-11 8.18732269e-02 1.12901500e-11
 1.17792961e-11 7.28749687e-02 1.95061990e-12 3.06567177e-01
 3.20787048e-11 7.72387651e-12 7.82427960e-12 9.40821616e-09
 3.67987990e-11 1.81350440e-11 1.22826548e-11 6.78436815e-02
 1.28838195e-01 3.95445351e-12]
Current bankroll: 79.92
Amount to bet: 13.32
Expected profit: 19.85 (149.04%)
Best case: 27.07 (203.21%)


,home_team,away_team,best_home_odds,best_home_bookmaker,best_away_odds,best_away_bookmaker,optimal_bet_home,optimal_bet_away,home_prediction,away_prediction
0,Washington Wizards,Chicago Bulls,1.80,DraftKings,2.10,PointsBet (US),0.00,2.66,0.287795,0.712205
1,Philadelphia 76ers,Orlando Magic,1.34,DraftKings,3.40,DraftKings,0.00,0.00,0.634578,0.365422
2,Boston Celtics,Charlotte Hornets,1.34,PointsBet (US),3.35,DraftKings,0.00,0.00,0.913023,0.086977
3,New York Knicks,Brooklyn Nets,1.16,DraftKings,5.55,DraftKings,0.00,1.86,0.636448,0.363552
4,Cleveland Cavaliers,Indiana Pacers,1.74,DraftKings,2.15,PointsBet (US),0.00,0.00,0.521351,0.478649
5,Minnesota Timberwolves,Atlanta Hawks,1.10,DraftKings,8.00,PointsBet (US),0.00,0.00,0.882435,0.117565
6,San Antonio Spurs,Denver Nuggets,5.10,DraftKings,1.20,PointsBet (US),0.00,0.00,0.224777,0.775223
7,Memphis Grizzlies,Los Angeles Lakers,9.00,PointsBet (US),1.08,PointsBet (US),1.07,0.00,0.230853,0.769147
8,Miami Heat,Toronto Raptors,1.10,PointsBet (US),8.00,DraftKings,0.00,0.93,0.762550,0.237450
9,Oklahoma City Thunder,Milwaukee Bucks,1.08,PointsBet (US),9.00,DraftKings,0.00,4.13,0.542385,0.457615


In [5]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = "1uUSEvXgQT3WZ6mSj6E0QVw48-B94E627HRo30LfJQpY"
SAMPLE_RANGE_NAME = "Sheet1!A2:I"

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


def append_values(spreadsheet_id, range_name, value_input_option, values):
  """
  Creates the batch_update the user has access to.
  Load pre-authorized user credentials from the environment.
  TODO(developer) - See https://developers.google.com/identity
  for guides on implementing OAuth2 for the application.
  """
  SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file"]
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
  # pylint: disable=maybe-no-member
  try:
    service = build("sheets", "v4", credentials=creds)
    body = {"values": values}
    result = (
        service.spreadsheets()
        .values()
        .append(
            spreadsheetId=spreadsheet_id,
            range=range_name,
            valueInputOption=value_input_option,
            body=body,
        )
        .execute()
    )
    print(f"{(result.get('updates').get('updatedCells'))} cells appended.")
    return result

  except HttpError as error:
    print(f"An error occurred: {error}")
    return error

def transform_team_name(team_name):
  name = team_name.split(' ')[-1]
  if name == 'Blazers':
    name = 'Trail Blazers'
  return name

def extract_values(bets):
    UPDATED_BANKROLL = """=INDIRECT("R[-1]C", "FALSE")-INDIRECT("R[-1]C[2]", "FALSE")+(IF(REGEXMATCH(INDIRECT("R[-1]C[4]", "FALSE"), "W"), INDIRECT("R[-1]C[3]", "FALSE"), 0))"""
    
    values = []
    # List home and away team bets as separate rows.  Do not include bets that are 0.
    for i in range(len(bets)):
        game = bets.iloc[i]
        if game['optimal_bet_home'] > 0:
            values.append([datetime.today().strftime('%Y/%m/%d'), transform_team_name(game['home_team']), transform_team_name(game['away_team']), 'Home', game['best_home_bookmaker'], UPDATED_BANKROLL, game['best_home_odds'], game['optimal_bet_home'],  """=ROUND(INDIRECT("RC[-2]", "FALSE")*INDIRECT("RC[-1]", "FALSE"), 2)""", "", game['home_prediction'], 1/game['best_home_odds']])
        if game['optimal_bet_away'] > 0:
            values.append([datetime.today().strftime('%Y/%m/%d'), transform_team_name(game['home_team']), transform_team_name(game['away_team']), 'Away', game['best_away_bookmaker'], UPDATED_BANKROLL, game['best_away_odds'], game['optimal_bet_away'], """=ROUND(INDIRECT("RC[-2]", "FALSE")*INDIRECT("RC[-1]", "FALSE"), 2)""", "", game['away_prediction'], 1/game['best_away_odds']])
    return values

values = extract_values(bets)
print(values)
append_values(SPREADSHEET_ID, SAMPLE_RANGE_NAME, "USER_ENTERED", values)

[['2024/04/12', 'Wizards', 'Bulls', 'Away', 'PointsBet (US)', '=INDIRECT("R[-1]C", "FALSE")-INDIRECT("R[-1]C[2]", "FALSE")+(IF(REGEXMATCH(INDIRECT("R[-1]C[4]", "FALSE"), "W"), INDIRECT("R[-1]C[3]", "FALSE"), 0))', 2.1, 2.66, '=ROUND(INDIRECT("RC[-2]", "FALSE")*INDIRECT("RC[-1]", "FALSE"), 2)', '', 0.7122054994106293, 0.47619047619047616], ['2024/04/12', 'Knicks', 'Nets', 'Away', 'DraftKings', '=INDIRECT("R[-1]C", "FALSE")-INDIRECT("R[-1]C[2]", "FALSE")+(IF(REGEXMATCH(INDIRECT("R[-1]C[4]", "FALSE"), "W"), INDIRECT("R[-1]C[3]", "FALSE"), 0))', 5.55, 1.86, '=ROUND(INDIRECT("RC[-2]", "FALSE")*INDIRECT("RC[-1]", "FALSE"), 2)', '', 0.3635520339012146, 0.1801801801801802], ['2024/04/12', 'Grizzlies', 'Lakers', 'Home', 'PointsBet (US)', '=INDIRECT("R[-1]C", "FALSE")-INDIRECT("R[-1]C[2]", "FALSE")+(IF(REGEXMATCH(INDIRECT("R[-1]C[4]", "FALSE"), "W"), INDIRECT("R[-1]C[3]", "FALSE"), 0))', 9.0, 1.07, '=ROUND(INDIRECT("RC[-2]", "FALSE")*INDIRECT("RC[-1]", "FALSE"), 2)', '', 0.23085343837738037, 0.1

{'spreadsheetId': '1uUSEvXgQT3WZ6mSj6E0QVw48-B94E627HRo30LfJQpY',
 'tableRange': 'Sheet1!A1:O422',
 'updates': {'spreadsheetId': '1uUSEvXgQT3WZ6mSj6E0QVw48-B94E627HRo30LfJQpY',
  'updatedRange': 'Sheet1!A423:L429',
  'updatedRows': 7,
  'updatedColumns': 12,
  'updatedCells': 84}}

In [6]:
# Identify value bets
value_bets = []
for game in best_odds.to_dict('records'):
    home_team = game['home_team']
    away_team = game['away_team']
    best_home_odds = game['best_home_odds']
    best_away_odds = game['best_away_odds']
    best_home_bookmaker = game['best_home_bookmaker']
    best_away_bookmaker = game['best_away_bookmaker']
    home_prediction = predictions[(home_team, away_team)]
    away_prediction = 1 - home_prediction
    # Compute which team has the higher expected value
    bet_on_home = True if home_prediction - 1/best_home_odds > away_prediction - 1/best_away_odds else False
    
    if bet_on_home and home_prediction > 1/best_home_odds:
        value_bets.append({
            'team': home_team,
            'odds': best_home_odds,
            'bookmaker': best_home_bookmaker,
            'probability': home_prediction,
            'implied_probability': 1/best_home_odds,
            'margin': home_prediction - 1/best_home_odds,
        })
    elif not bet_on_home and away_prediction > 1/best_away_odds:
        value_bets.append({
            'team': away_team,
            'odds': best_away_odds,
            'bookmaker': best_away_bookmaker,
            'probability': away_prediction,
            'implied_probability': 1/best_away_odds,
            'margin': away_prediction - 1/best_away_odds,
        })
value_bets = pd.DataFrame(value_bets)
value_bets.sort_values(by='margin', ascending=False, inplace=True)
value_bets.reset_index(inplace=True, drop=True)
value_bets

,team,odds,bookmaker,probability,implied_probability,margin
0,Milwaukee Bucks,9.00,DraftKings,0.457615,0.111111,0.346504
1,Chicago Bulls,2.10,PointsBet (US),0.712205,0.476190,0.236015
2,Sacramento Kings,2.70,DraftKings,0.560906,0.370370,0.190536
3,Brooklyn Nets,5.55,DraftKings,0.363552,0.180180,0.183372
4,Boston Celtics,1.34,PointsBet (US),0.913023,0.746269,0.166754
5,Memphis Grizzlies,9.00,PointsBet (US),0.230853,0.111111,0.119742
6,Toronto Raptors,8.00,DraftKings,0.237450,0.125000,0.112450
7,New Orleans Pelicans,2.50,PointsBet (US),0.510801,0.400000,0.110801
8,Utah Jazz,9.00,PointsBet (US),0.215867,0.111111,0.104756
9,Orlando Magic,3.40,DraftKings,0.365422,0.294118,0.071304


In [7]:
from utils import get_injury_report

injury_report = get_injury_report()
for i in range(len(injury_report)):
    player = injury_report.iloc[i]
    print(f'{player["TEAM"]} - {player["NAME"]} - {player["STATUS"]}')

Could not find player Jaylen Clark
Could not find player D.J. Carton
Atlanta Hawks - Clint Capela - Day-To-Day
Atlanta Hawks - Wesley Matthews - Out
Atlanta Hawks - Seth Lundy - Out
Atlanta Hawks - AJ Griffin - Out
Atlanta Hawks - Dejounte Murray - Day-To-Day
Atlanta Hawks - Onyeka Okongwu - Out
Atlanta Hawks - Jalen Johnson - Out
Atlanta Hawks - Saddiq Bey - Out
Boston Celtics - Jrue Holiday - Day-To-Day
Boston Celtics - Jaylen Brown - Day-To-Day
Boston Celtics - Kristaps Porzingis - Day-To-Day
Boston Celtics - Jayson Tatum - Day-To-Day
Boston Celtics - Oshae Brissett - Day-To-Day
Brooklyn Nets - Day'Ron Sharpe - Day-To-Day
Brooklyn Nets - Jaylen Martin - Out
Brooklyn Nets - Jacob Gilyard - Day-To-Day
Brooklyn Nets - Dorian Finney-Smith - Out
Brooklyn Nets - Cameron Johnson - Day-To-Day
Brooklyn Nets - Dennis Smith Jr. - Out
Brooklyn Nets - Keita Bates-Diop - Out
Brooklyn Nets - Ben Simmons - Out
Brooklyn Nets - Dariq Whitehead - Out
Chicago Bulls - Ayo Dosunmu - Day-To-Day
Chicago Bu